In [5]:
import subprocess
import sys
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "hydra-core"])

import hydra
from hydra import compose, initialize_config_dir
from omegaconf import OmegaConf, DictConfig
from dataclasses import dataclass, field
from typing import List, Optional
import os
from pathlib import Path

In [6]:
@dataclass
class OptimizerConfig:
    _target_: str = "torch.optim.SGD"
    lr: float = 0.01

@dataclass
class AdamConfig(OptimizerConfig):
    _target_: str = "torch.optim.Adam"
    lr: float = 0.001
    betas: tuple = (0.9, 0.999)
    weight_decay: float = 0.0

@dataclass
class SGDConfig(OptimizerConfig):
    _target_: str = "torch.optim.SGD"
    lr: float = 0.01
    momentum: float = 0.9
    nesterov: bool = True

@dataclass
class ModelConfig:
    name: str = "resnet"
    num_layers: int = 50
    hidden_dim: int = 512
    dropout: float = 0.1

@dataclass
class DataConfig:
    dataset: str = "cifar10"
    batch_size: int = 32
    num_workers: int = 4
    augmentation: bool = True

@dataclass
class TrainingConfig:
    model: ModelConfig = field(default_factory=ModelConfig)
    data: DataConfig = field(default_factory=DataConfig)
    optimizer: OptimizerConfig = field(default_factory=AdamConfig)
    epochs: int = 100
    seed: int = 42
    device: str = "cuda"
    experiment_name: str = "exp_001"

In [7]:
def setup_config_dir():
    config_dir = Path("./hydra_configs")
    config_dir.mkdir(exist_ok=True)

    main_config = """
defaults:
  - model: resnet
  - data: cifar10
  - optimizer: adam
  - _self_

epochs: 100
seed: 42
device: cuda
experiment_name: exp_001
"""
    (config_dir / "config.yaml").write_text(main_config)

    model_dir = config_dir / "model"
    model_dir.mkdir(exist_ok=True)

    (model_dir / "resnet.yaml").write_text("""
name: resnet
num_layers: 50
hidden_dim: 512
dropout: 0.1
""")

    (model_dir / "vit.yaml").write_text("""
name: vision_transformer
num_layers: 12
hidden_dim: 768
dropout: 0.1
patch_size: 16
""")

    data_dir = config_dir / "data"
    data_dir.mkdir(exist_ok=True)

    (data_dir / "cifar10.yaml").write_text("""
dataset: cifar10
batch_size: 32
num_workers: 4
augmentation: true
""")

    (data_dir / "imagenet.yaml").write_text("""
dataset: imagenet
batch_size: 128
num_workers: 8
augmentation: true
""")

    opt_dir = config_dir / "optimizer"
    opt_dir.mkdir(exist_ok=True)

    (opt_dir / "adam.yaml").write_text("""
_target_: torch.optim.Adam
lr: 0.001
betas: [0.9, 0.999]
weight_decay: 0.0
""")

    (opt_dir / "sgd.yaml").write_text("""
_target_: torch.optim.SGD
lr: 0.01
momentum: 0.9
nesterov: true
""")

    return str(config_dir.absolute())

In [8]:
@hydra.main(version_base=None, config_path="hydra_configs", config_name="config")
def train(cfg: DictConfig) -> float:
    print("=" * 80)
    print("CONFIGURATION")
    print("=" * 80)
    print(OmegaConf.to_yaml(cfg))

    print("\n" + "=" * 80)
    print("ACCESSING CONFIGURATION VALUES")
    print("=" * 80)
    print(f"Model: {cfg.model.name}")
    print(f"Dataset: {cfg.data.dataset}")
    print(f"Batch Size: {cfg.data.batch_size}")
    print(f"Optimizer LR: {cfg.optimizer.lr}")
    print(f"Epochs: {cfg.epochs}")

    best_acc = 0.0
    for epoch in range(min(cfg.epochs, 3)):
        acc = 0.5 + (epoch * 0.1) + (cfg.optimizer.lr * 10)
        best_acc = max(best_acc, acc)
        print(f"Epoch {epoch+1}/{cfg.epochs}: Accuracy = {acc:.4f}")

    return best_acc

In [9]:
def demo_basic_usage():
    print("\n" + "🚀 DEMO 1: Basic Configuration\n")
    config_dir = setup_config_dir()
    with initialize_config_dir(version_base=None, config_dir=config_dir):
        cfg = compose(config_name="config")
        print(OmegaConf.to_yaml(cfg))

def demo_config_override():
    print("\n" + "🚀 DEMO 2: Configuration Overrides\n")
    config_dir = setup_config_dir()
    with initialize_config_dir(version_base=None, config_dir=config_dir):
        cfg = compose(
            config_name="config",
            overrides=[
                "model=vit",
                "data=imagenet",
                "optimizer=sgd",
                "optimizer.lr=0.1",
                "epochs=50"
            ]
        )
        print(OmegaConf.to_yaml(cfg))

def demo_structured_config():
    print("\n" + "🚀 DEMO 3: Structured Config Validation\n")
    from hydra.core.config_store import ConfigStore
    cs = ConfigStore.instance()
    cs.store(name="training_config", node=TrainingConfig)
    with initialize_config_dir(version_base=None, config_dir=setup_config_dir()):
        cfg = compose(config_name="config")
        print(f"Config type: {type(cfg)}")
        print(f"Epochs (validated as int): {cfg.epochs}")

def demo_multirun_simulation():
    print("\n" + "🚀 DEMO 4: Multirun Simulation\n")
    config_dir = setup_config_dir()
    experiments = [
        ["model=resnet", "optimizer=adam", "optimizer.lr=0.001"],
        ["model=resnet", "optimizer=sgd", "optimizer.lr=0.01"],
        ["model=vit", "optimizer=adam", "optimizer.lr=0.0001"],
    ]
    results = {}
    for i, overrides in enumerate(experiments):
        print(f"\n--- Experiment {i+1} ---")
        with initialize_config_dir(version_base=None, config_dir=config_dir):
            cfg = compose(config_name="config", overrides=overrides)
            print(f"Model: {cfg.model.name}, Optimizer: {cfg.optimizer._target_}")
            print(f"Learning Rate: {cfg.optimizer.lr}")
            results[f"exp_{i+1}"] = cfg
    return results

def demo_interpolation():
    print("\n" + "🚀 DEMO 5: Variable Interpolation\n")
    cfg = OmegaConf.create({
        "model": {"name": "resnet", "layers": 50},
        "experiment": "${model.name}_${model.layers}",
        "output_dir": "/outputs/${experiment}",
        "checkpoint": "${output_dir}/best.ckpt"
    })
    print(OmegaConf.to_yaml(cfg))
    print(f"\nResolved experiment name: {cfg.experiment}")
    print(f"Resolved checkpoint path: {cfg.checkpoint}")

In [10]:
if __name__ == "__main__":
    demo_basic_usage()
    demo_config_override()
    demo_structured_config()
    demo_multirun_simulation()
    demo_interpolation()
    print("\n" + "=" * 80)
    print("Tutorial complete! Key takeaways:")
    print("✓ Config composition with defaults")
    print("✓ Runtime overrides via command line")
    print("✓ Structured configs with type safety")
    print("✓ Multirun for hyperparameter sweeps")
    print("✓ Variable interpolation")
    print("=" * 80)


🚀 DEMO 1: Basic Configuration

model:
  name: resnet
  num_layers: 50
  hidden_dim: 512
  dropout: 0.1
data:
  dataset: cifar10
  batch_size: 32
  num_workers: 4
  augmentation: true
optimizer:
  _target_: torch.optim.Adam
  lr: 0.001
  betas:
  - 0.9
  - 0.999
  weight_decay: 0.0
epochs: 100
seed: 42
device: cuda
experiment_name: exp_001


🚀 DEMO 2: Configuration Overrides

model:
  name: vision_transformer
  num_layers: 12
  hidden_dim: 768
  dropout: 0.1
  patch_size: 16
data:
  dataset: imagenet
  batch_size: 128
  num_workers: 8
  augmentation: true
optimizer:
  _target_: torch.optim.SGD
  lr: 0.1
  momentum: 0.9
  nesterov: true
epochs: 50
seed: 42
device: cuda
experiment_name: exp_001


🚀 DEMO 3: Structured Config Validation

Config type: <class 'omegaconf.dictconfig.DictConfig'>
Epochs (validated as int): 100

🚀 DEMO 4: Multirun Simulation


--- Experiment 1 ---
Model: resnet, Optimizer: torch.optim.Adam
Learning Rate: 0.001

--- Experiment 2 ---
Model: resnet, Optimizer: torc